In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import requests
from config import eia_key

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import xgboost as xgb


*Electricity Demand Data*

create your api account on https://www.eia.gov/opendata/ and get own your API key.
save that api ket with eia_key variable in this file before running following code.

In [1]:
#eia_key = "your_api_key"

In [2]:
offset = 0
df = []
while offset<48900:
    url = 'https://api.eia.gov/v2/electricity/rto/region-data/data/?frequency=hourly&data[0]=value&facets[respondent][]=NY&facets[type][]=D&start=2019-01-01T00&end=2024-08-01T00&sort[0][column]=period&sort[0][direction]=desc&offset=' + str(offset) + "&length=5000&api_key=" + eia_key
    data = requests.get(url).json()['response']['data']
    data = pd.DataFrame(data)
    df.append(data)
    offset+=5000
    

In [3]:
data = pd.concat(df, ignore_index=True)


demand_hourly = data[['period', 'value']].rename(columns={'period': 'date', 'value': 'demand'})
demand_hourly['date'] = pd.to_datetime(demand_hourly['date'], infer_datetime_format=True)


/tmp/ipykernel_33782/1324632485.py:5: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  demand_hourly['date'] = pd.to_datetime(demand_hourly['date'], infer_datetime_format=True)


Temperature Data

In [4]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 40.7143,
	"longitude": -74.006,
	"start_date": "2019-01-01",
	"end_date": "2024-08-01",
	"hourly": "temperature_2m",
	"timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature"] = hourly_temperature_2m

hourly_temperature_dataframe = pd.DataFrame(data = hourly_data)

hourly_temperature_dataframe['date'] = hourly_temperature_dataframe['date'].dt.tz_localize(None).dt.strftime('%Y-%m-%d %H:%M:%S')


Data Preprocessing

In [5]:


# Assuming demand_hourly and hourly_temperature_dataframe are your two dataframes

# Convert the date columns to datetime format if they are not already
demand_hourly['date'] = pd.to_datetime(demand_hourly['date'])
hourly_temperature_dataframe['date'] = pd.to_datetime(hourly_temperature_dataframe['date'])

# Ensure the datetime values are floored to the nearest hour for consistency
demand_hourly['date'] = demand_hourly['date'].dt.floor('H')
hourly_temperature_dataframe['date'] = hourly_temperature_dataframe['date'].dt.floor('H')

# Sort both dataframes by date
demand_hourly.sort_values('date', inplace=True)
hourly_temperature_dataframe.sort_values('date', inplace=True)

# Merge the dataframes on the 'date' column
df = pd.merge(hourly_temperature_dataframe, demand_hourly, on='date', how='inner')
df.to_csv('dataset.csv')
# Display the combined dataframe
print(df)


                     date  temperature demand
0     2019-01-01 04:00:00     7.872500  16613
1     2019-01-01 05:00:00     8.672500  15774
2     2019-01-01 06:00:00     9.172500  15053
3     2019-01-01 07:00:00    10.822500  14481
4     2019-01-01 08:00:00    12.572500  13927
...                   ...          ...    ...
48928 2024-07-31 20:00:00    31.822498  26673
48929 2024-07-31 21:00:00    30.722500  27069
48930 2024-07-31 22:00:00    25.972500  27552
48931 2024-07-31 23:00:00    25.722500  27566
48932 2024-08-01 00:00:00    24.372499  27194

[48933 rows x 3 columns]


/tmp/ipykernel_33782/1378415741.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  demand_hourly['date'] = demand_hourly['date'].dt.floor('H')
/tmp/ipykernel_33782/1378415741.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_temperature_dataframe['date'] = hourly_temperature_dataframe['date'].dt.floor('H')


In [6]:
# Extract features from 'date' column
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hr'] = df['date'].dt.hour
df['day_of_week'] = df['date'].dt.dayofweek  # Monday=0, Sunday=6
df['is_weekend'] = df['date'].dt.dayofweek >= 5  # True if weekend, False otherwise

In [7]:
#  bank holiday
holidays = calendar().holidays(start=df['date'].min(), end=df['date'].max())
df['holiday'] = df['date'].isin(holidays).astype(int)
df['date'] = pd.to_datetime(df['date'])

df['demand'] = pd.to_numeric(df['demand'], errors='coerce').astype('float')
print(df.dtypes)


date           datetime64[ns]
temperature           float32
demand                float64
year                    int32
month                   int32
day                     int32
hr                      int32
day_of_week             int32
is_weekend               bool
holiday                 int64
dtype: object


In [10]:
df.mean()

date           2021-10-16 14:00:00
temperature              12.405856
demand                17294.824433
year                   2021.313142
month                     6.261378
day                       15.72822
hr                       11.500582
day_of_week               2.998692
is_weekend                0.285452
holiday                   0.001185
dtype: object

Splitting the Dataset

In [8]:
# Define features and target
X = df.drop('demand', axis=1)
X = X.drop('date', axis=1)

y = df['demand']

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Model Training

In [51]:

# Define the model
model = xgb.XGBRegressor(tree_method="hist", early_stopping_rounds=3)

# Train the model
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 1117.570812080609


In [52]:

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R² Score: {r2}')

Mean Squared Error: 1248964.5200145114
Mean Absolute Error: 794.1748928944327
R² Score: 0.8741991854817652


In [119]:
new_observation = pd.DataFrame({
    'temperature' : [21],
    'year': [2024],
    'month': [8],
    'day': [12],
    'hr': [3],
    'day_of_week': [1],
    'is_weekend': [0],
    'holiday': [0]
})

In [120]:
y_pred = model.predict(new_observation)

In [121]:
print(y_pred)

[20032.707]


In [74]:
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, tree_method="hist", random_state=42)
}

# Evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_test, y_pred)
    print(f'{name} - Root Mean Squared Error: {rmse}, R^2 Score: {r2}')


Linear Regression - Root Mean Squared Error: 2853.410496488196, R^2 Score: 0.1799093495395423
Random Forest - Root Mean Squared Error: 700.6667716579755, R^2 Score: 0.9505511272433568
Gradient Boosting - Root Mean Squared Error: 958.3306321314395, R^2 Score: 0.9074952375822367
XGBoost - Root Mean Squared Error: 716.2864835682041, R^2 Score: 0.9483218612325548
